# DMA AXI Stream FIFO Test for PYNQ

# DMA Master connected to S_AXI_HP2 Port of Zynq System via AXI Interconnect (axi_mem_intercon) used by tracebufer_pmods/audrino
#DMA S_AXI_LITE port connected to AXI Interconnect (processing_system7_0_axi_periph)
#System Block diagram at: https://github.com/IamVNIE/hapi-des/blob/master/notebook/axi-stream-fifo-test/pynq.xps
#DMA with FIFO created using: http://www.fpgadeveloper.com/2014/08/using-the-axi-dma-in-vivado.html
#Thanks to Peter Ogden at Xilinx for the solution for DMA buffer accesses from Python

In [14]:
from pynq import Overlay
ol = Overlay("axi_stream.bit")
#ol = Overlay("axis_inc_test.bit")

ol.download()
ol.bitstream.timestamp

from pynq import PL
PL.bitfile_name

ol.is_loaded()

True

# Testing dma class provided in Base PYNQ  

In [15]:
from pynq.drivers.dma import DMA
dma_base = int(PL.ip_dict["SEG_axi_dma_0_Reg2"][0],16)
print(hex(dma_base))
TXdma = DMA(0x40400000, direction=0)
RXdma = DMA(0x40400000, direction=1)
#---- GET Data from PL ---#
print(TXdma.Configuration)
print(RXdma.Configuration)

0x40400000
phyAddress = 0x40400000
phyAddress = 0x40400000
{'Mm2SDataWidth': 32, 'S2MmDataWidth': 64, 'DeviceId': 5, 'HasStsCntrlStrm': 0, 'MicroDmaMode': 0, 'BaseAddr': <cdata 'uint32_t *' 0x30adc000>, 'Mm2SBurstSize': 16, 'S2MmNumChannels': 1, 'HasMm2S': 1, 'HasMm2SDRE': 0, 'S2MmBurstSize': 64, 'HasS2Mm': 0, 'HasSg': 0, 'AddrWidth': 32, 'Mm2sNumChannels': 1, 'HasS2MmDRE': 0}
{'Mm2SDataWidth': 32, 'S2MmDataWidth': 64, 'DeviceId': 6, 'HasStsCntrlStrm': 0, 'MicroDmaMode': 0, 'BaseAddr': <cdata 'uint32_t *' 0x30acc000>, 'Mm2SBurstSize': 16, 'S2MmNumChannels': 1, 'HasMm2S': 0, 'HasMm2SDRE': 0, 'S2MmBurstSize': 64, 'HasS2Mm': 1, 'HasSg': 0, 'AddrWidth': 32, 'Mm2sNumChannels': 1, 'HasS2MmDRE': 0}


In [16]:
import numpy as np
from random import randint
import time
from pynq.drivers.dma import ffi

#data2Transmit=[randint(0,x) for x in range(50000) ]
data2Transmit=[1,2,3]
size=int(len(data2Transmit)/2*8)
TXdma.create_buf(size)
RXdma.create_buf(size)
buf = ffi.buffer(TXdma.buf, size)
view = np.frombuffer(buf, np.int32,-1)
print(view)
view[:] = data2Transmit
result = view.copy()
print(result)

# Sending data from PS-PL
tx_strt = time.time()
dma.transfer(size,direction=0)
#dma.wait()  #-- Waiting for transfer to complete always times out with error
tx_done=time.time()
# Recieve data from PL-PS
rx_strt = time.time()
RXdma.transfer(size,direction=1) 
rx_done=time.time()
TXdma.wait()
RXdma.wait()

bufPhyAddr  0x1684a000
bufPhyAddr  0x1684b000
[0 0 0]
[1 2 3]
direction = 0
direction = 1


TimeoutError: DMA wait timed out.

In [17]:
# Check if transmitted values are same as recieved values (Its just a FIFO attached to DMA)
vin = RXdma.get_buf()

dataRecieved=[]

for i in range(len(view)):
    dataRecieved.append(vin[i])
print(dataRecieved[0:10])


[0, 0, 0]


In [18]:
## ----For Statistics --- ##
rx= rx_done - rx_strt
tx= tx_done - tx_strt
txLen=len(data2Transmit)*32
txbps=len(data2Transmit)*32/tx
rxbps=len(data2Transmit)*32/rx
txMbps=(len(data2Transmit)*32/tx)/1e6
rxMbps=(len(data2Transmit)*32/rx)/1e6
txGbps=(len(data2Transmit)*32/tx)/1e9
rxGbps=(len(data2Transmit)*32/rx)/1e9
print("TOTAL TIME TO SEND - ",txLen, "BITS = ", tx)
print("TOTAL TIME TO RECV - ",txLen, "BITS = ", rx)
print("TX BW = ",txbps, "bps | ",txMbps,"Mbps | ",txGbps,"Gbps")
print("RX BW = ",rxbps, "bps | ",rxMbps,"Mbps | ",rxGbps,"Gbps")
print("TOTAL BW = ",txMbps+rxMbps,"Mbps | ",txGbps+rxGbps,"Gbps")


TOTAL TIME TO SEND -  96 BITS =  0.0010361671447753906
TOTAL TIME TO RECV -  96 BITS =  0.000934600830078125
TX BW =  92649.14496088357 bps |  0.09264914496088357 Mbps |  9.264914496088357e-05 Gbps
RX BW =  102717.64897959183 bps |  0.10271764897959183 Mbps |  0.00010271764897959183 Gbps
TOTAL BW =  0.1953667939404754 Mbps |  0.00019536679394047541 Gbps
